# **파트 5: 머신러닝 인과추론 방법론**

---

# 29. 머신러닝 인과추론 핵심 아이디어: 두 세계의 결합

---

**머신러닝 인과추론(Machine Learning Causal Inference, MLCI)**은 우리가 지금까지 쌓아온 두 거대한 산맥의 지식을 하나로 합치는 것이다.

1.  **'대주제 2: 전통적 인과추론'의 세계**:
    - **목표**: 인과관계를 식별하기 위한 엄밀한 **'논리적 프레임워크'** (예: OVB를 어떻게 해결할 것인가?)
    - **한계**: 주로 선형 모델을 가정하며, 다룰 수 있는 통제변수의 수가 제한적이다. 현실의 복잡한 데이터 패턴을 놓칠 수 있다.

2.  **'대주제 4: 머신러닝 예측'의 세계**:
    - **목표**: 고차원의 복잡한 데이터를 다루는 **'강력한 예측 엔진'** (예: 어떻게 하면 Y를 가장 정확하게 맞출 것인가?)
    - **한계**: 예측 성능 자체에만 집중하므로, 모델이 '블랙박스'처럼 작동한다. 특정 변수의 '인과적 효과'를 편향 없이 추정하는 것을 보장하지 않는다.

**MLCI의 최종 목표**: 전통적 인과추론의 **'엄밀한 논리'**는 그대로 가져오되, 그 계산 엔진을 머신러닝의 **'강력한 예측력'**으로 교체하여, 전통적인 방법으로는 불가능했던 복잡하고 현실적인 문제 상황에서 신뢰도 높은 인과 효과를 추정하는 것입니다.

<br>

### **핵심 과제: 어떻게 두 세계를 결합하는가? (The Division of Labor)**

머신러닝 인과추론의 핵심 철학은 **'역할 분담'**입니다. 인과 효과 추정 문제를 두 가지로 나눈다.

1.  **우리가 정말 알고 싶은 단 하나의 '관심 대상'**:
    - 정책의 순수한 인과 효과 ($\tau$)

2.  **그것을 알기 위해 어쩔 수 없이 추정해야 하는 '성가신 부분 (Nuisance Functions)'**:
    - 통제변수($X$)가 결과($Y$)에 미치는 복잡한 관계 ($E[Y|X]$)
    - 통제변수($X$)가 정책($D$)에 미치는 복잡한 관계 ($E[D|X]$, 즉 성향점수)

**역할 분담의 원리**:
- **머신러닝의 역할**: 인간이 파악하기 힘든, 복잡하고 고차원적인 **'성가신 부분'의 예측**을 전담합니다. 라쏘, 랜덤 포레스트, 부스팅 등 강력한 예측 모델이 이 역할을 수행한다.
- **계량경제학(인과추론)의 역할**: 머신러닝이 예측 과정에서 만들어내는 약간의 편향(bias)들이, 우리가 정말 원하는 **'관심 대상'($\tau$)을 오염시키지 않도록 막아주는 정교한 '통계적 안전장치'**를 설계한다. (예: DML의 직교성(orthogonality)과 교차적합(cross-fitting))

**비유**:
- **인과추론 프레임워크**: 정밀한 종양 제거 수술을 집도하는 **'외과 의사'**. 최종 목표는 오직 종양($\tau$)을 정확히 제거하는 것이다.
- **머신러닝 모델**: 수술 중 환자의 혈압, 산소포화도 등 수백 가지 생체 신호를 실시간으로 분석하고 예측하는 **'최첨단 의료 로봇'**.
- **MLCI**: 외과 의사는 의료 로봇에게 모든 복잡한 '주변부 관리'를 맡기고, 자신은 가장 중요한 '핵심 수술'에만 집중한다. 로봇의 강력한 성능 덕분에, 과거에는 불가능했던 복잡한 환경에서도 정밀한 수술이 가능해진다.

<br>

### **이 파트의 주요 목표들**
- **평균 처치 효과(ATE)의 확장**: 수백, 수천 개의 통제변수가 있는 상황에서도 정책의 평균 효과를 정확하게 추정한다. (→ **DML**)
- **이질적 처치 효과(HTE)의 발견**: "누구에게 효과가 더 큰가?"라는 질문에, 복잡한 데이터 속에서 머신러닝을 이용해 답을 찾는다. (→ **Meta-Learners, 인과 포레스트**)
- **전통적 방법론의 강화**: 도구변수법(IV)과 같은 전통적 설계에 머신러닝을 접목하여 비선형 관계를 다룬다. (→ **IV with ML**)




# 30. 이중/편향제거 머신러닝 (Double/Debiased Machine Learning, DML)

---

### **핵심 아이디어**
- **목표**: 수백, 수천 개의 매우 많은 통제변수($X$)가 있는 고차원(high-dimensional) 상황에서, 우리가 정말 관심 있는 단 하나의 변수(처치 $D$)가 결과($Y$)에 미치는 **평균적인 인과 효과($\tau$)**를 편향 없이 추정하는 것.
- **문제의식**:
    1.  전통적인 OLS는 이렇게 많은 변수를 다룰 때 과적합이나 다중공선성 문제에 시달린다.
    2.  라쏘(Lasso)와 같은 머신러닝 예측 모델은 변수 선택과 예측은 잘하지만, 그 자체로 인과 효과($\tau$)를 편향 없이 추정하도록 설계되지 않았다. (오히려 변수 선택 과정에서 $\tau$에 편향을 유발할 수 있다.)
- **DML의 해법**: **'두 번의 머신러닝 예측'**과 **'잔차 활용'**이라는 절차를 통해, 머신러닝의 강력한 예측력을 마음껏 활용하면서도, 우리가 원하는 인과 효과($\tau$)에 대한 추정치는 깨끗하게 유지하는 매우 영리한 방법론이다.

<br>

### **DML의 작동 원리 (매우 중요!)**
DML은 '나이만-직교성(Neyman Orthogonality)'이라는 통계적 원리를 기반으로 하며, 다음과 같은 3단계로 작동한다. (교차적합(Cross-fitting) 포함)

#### **1단계: Y에 대한 예측 (Outcome Regression)**
- 처치 변수($D$)를 제외하고, 오직 **모든 통제변수($X$)만을 사용**하여 결과 변수($Y$)를 예측하는 머신러닝 모델을 만든다. (이때 라쏘, 랜덤 포레스트 등 어떤 모델이든 사용 가능)
- 이 모델을 이용해 $Y$의 예측값($\hat{Y}$)을 구한 뒤, **실제값과의 차이(잔차)**를 계산한다.
    - $\tilde{Y} = Y - \hat{Y}$
- **직관적 의미**: 이 잔차 $\tilde{Y}$는, 수많은 통제변수 $X$들로는 **'설명되지 않는 순수한 Y의 변동분'**을 의미한다.

#### **2단계: D에 대한 예측 (Treatment Regression)**
- 결과 변수($Y$)는 신경 쓰지 않고, **모든 통제변수($X$)만을 사용**하여 처치 변수($D$)를 예측하는 머신러닝 모델을 또 하나 만든다.
- 이 모델을 이용해 $D$의 예측값($\hat{D}$)을 구한 뒤, **실제값과의 차이(잔차)**를 계산한다.
    - $\tilde{D} = D - \hat{D}$
- **직관적 의미**: 이 잔차 $\tilde{D}$는, 수많은 통제변수 $X$들과는 **'상관없는 순수한 D의 변동분'**을 의미한다.

#### **3단계: 잔차에 대한 회귀분석 (Final Stage Regression)**
- 마지막으로, 1단계에서 구한 **'순수한 Y의 변동분($\tilde{Y}$)'**을 2단계에서 구한 **'순수한 D의 변동분($\tilde{D}$)'**에 OLS 회귀분석한다.
    $$
    \tilde{Y} = \tau \tilde{D} + \text{error}
    $$
- **결론**: 이 마지막 회귀분석에서 얻은 계수 **$\hat{\tau}$가 바로 우리가 찾던, 편향이 제거된 인과 효과 추정치**이다. $X$의 영향력이 양쪽 변수($Y, D$)에서 모두 '청소'되었기 때문에, 이 둘의 관계는 $X$라는 교란 변수(confounder)의 영향으로부터 자유로워진다.

<br>

### **핵심 장치: 교차적합 (Cross-fitting)**
- 위 3단계를 더 정교하게 만들기 위해, DML은 교차적합이라는 장치를 사용한다.
- **방법**: 데이터를 무작위로 두 그룹(A, B)으로 나눈다.
    1. **B 그룹 데이터**를 사용해서 1, 2단계의 예측 모델들을 '학습'시킨다.
    2. 학습된 모델을 **A 그룹 데이터**에 '적용'하여 잔차($\tilde{Y}_A, \tilde{D}_A$)를 계산한다.
    3. 역할을 바꿔서, **A 그룹 데이터**로 모델을 '학습'시키고, **B 그룹 데이터**에 '적용'하여 잔차($\tilde{Y}_B, \tilde{D}_B$)를 계산한다.
    4. 이렇게 얻은 전체 잔차를 합쳐서 3단계 최종 회귀분석을 실행한다.
- **목적**: 자기 자신이 포함된 데이터로 학습한 모델로 자기 자신을 예측할 때 발생하는 미세한 과적합 편향(overfitting bias)마저도 제거하여, 추정치의 통계적 성질을 더욱 좋게 만들기 위한 기술적 장치이다.

<br>

### **결론**
- DML은 전통적 인과추론의 핵심 문제의식(OVB)과 이중 강건 추정법의 아이디어를 머신러닝 시대에 맞게 구현한 **현대 인과추론의 표준적인 출발점**이다.
- 연구자는 복잡한 통제변수 처리 문제를 강력한 머신러닝 알고리즘에 맡기고, 오직 핵심적인 인과 효과($\tau$)에만 집중할 수 있게 해준다.

# 31. 머신러닝을 활용한 도구변수법 (IV with Machine Learning)

---

### **핵심 아이디어**
- 전통적인 도구변수법(IV), 특히 2단계 최소자승법(2SLS)은 각 단계의 관계가 **'선형(linear)'**이라고 가정한다.
    - 1단계: 도구변수($Z$)가 내생변수($X$)에 미치는 영향이 선형적이다.
    - 2단계: '정화된' 내생변수($\hat{X}$)가 결과($Y$)에 미치는 영향이 선형적이다.
- 하지만 현실에서는 이 관계들이 매우 **복잡한 비선형(non-linear)** 형태일 수 있다.
- **머신러닝(ML)을 활용한 IV**는, 2SLS의 각 단계를 OLS 대신 **예측력이 뛰어난 머신러닝 모델(랜덤 포레스트, 신경망 등)**로 대체하여, 이러한 복잡한 비선형 관계를 유연하게 잡아내려는 시도이다.
- **비유**: 전통적인 2SLS가 '자'를 이용해 변수들의 관계를 직선으로 근사했다면, ML을 활용한 IV는 '프렌치 커브'나 '3D 스캐너'를 이용해 그 관계를 훨씬 더 정교하고 정확하게 그려내는 것과 같다.

<br>

### **어디에 머신러닝을 사용하는가?: 2SLS의 각 단계**

#### **1. 1단계(First Stage)의 강화: 더 정교한 예측**
- **전통적 방식**: $X$를 $Z$와 통제변수들에 대해 OLS 회귀분석을 실행한다.
    - $$ X = \pi_0 + \pi_1 Z + \text{controls} + \nu $$
- **ML 활용**: 이 관계가 비선형일 가능성을 고려하여, OLS 대신 **라쏘, 랜덤 포레스트, 부스팅, 신경망** 등 예측력이 뛰어난 ML 모델을 사용해 $X$의 예측값 $\hat{X}$를 구한다.
    - $$ \hat{X} = \text{ML_model}(Z, \text{controls}) $$
- **장점**: 1단계 예측이 더 정확해지면, '정화된' 변수 $\hat{X}$가 더 많은 유효한 정보를 담게 되어, 최종적으로 2단계에서 얻는 인과 효과 추정치의 **정밀도(precision)와 통계적 검정력(power)이 향상**될 수 있다.

#### **2. 2단계(Second Stage)의 강화: 이질적 효과 탐색**
- **전통적 방식**: $Y$를 1단계에서 얻은 $\hat{X}$에 대해 OLS 회귀분석을 실행한다.
    - $$ Y = \beta_0 + \beta_1 \hat{X} + \text{controls} + \epsilon $$
    - 여기서 $\beta_1$은 **평균적인(average)** 인과 효과를 나타낸다.
- **ML 활용**: 2단계 역시 복잡한 비선형 관계를 가질 수 있다. 더 나아가, $X$가 $Y$에 미치는 효과가 **개인의 특성에 따라 달라지는 '이질적 처치 효과(HTE)'**일 수 있다.
- 이 단계에서 머신러닝을 활용하면, 이러한 **비선형적이고 이질적인 인과 효과**를 추정할 수 있다.

<br>

### **주요 현대적 IV 방법론**

1.  **Deep IV (신경망을 이용한 도구변수법)**
    - **핵심**: 1단계와 2단계 모두에 **딥러닝(Deep Neural Networks)**을 사용하는 방법.
    - **특징**: 변수들 간의 관계가 매우 복잡하고 데이터가 방대할 때 강력한 성능을 보인다. 특히 처치 변수 $X$가 연속형이거나 매우 복잡한 형태일 때 유용하다.

2.  **Causal Forest for IV (인과 포레스트를 이용한 도구변수법)**
    - **핵심**: '인과 포레스트'의 아이디어를 IV 설정으로 확장한 것.
    - **목표**: $X$가 $Y$에 미치는 **'평균적인' 효과(ATE)**를 넘어, 개인의 특성에 따라 그 효과가 어떻게 달라지는지(**'이질적인' 효과, CATE**)를 도구변수를 이용해 추정한다.

<br>

### **결론 및 주의사항**
- **장점**:
    - **유연성**: 선형성이라는 강력한 가정을 완화하여, 현실의 복잡한 데이터 패턴을 더 잘 반영할 수 있다.
    - **정밀도 향상**: 1단계 예측력을 높여 최종 추정치의 효율성을 개선할 수 있다.
- **매우 중요한 주의사항**:
    - 머신러닝을 사용한다고 해서, 도구변수법의 **근본적인 두 가지 가정(연관성, 배제 제약)이 면제되는 것은 절대 아니다.**
    - 즉, $Z$는 여전히 $X$와 강하게 연관되어야 하고($Cov(Z,X) \neq 0$), 오직 $X$를 통해서만 $Y$에 영향을 미쳐야 한다($Cov(Z,u) = 0$).
    - ML은 **'추정(Estimation)'**의 문제를 도와주는 것이지, **'식별(Identification)'**의 문제를 해결해주는 마법 지팡이가 아니다. 연구 설계의 타당성은 여전히 연구자의 몫이다.

# 32. Meta-Learners: 이질적 처치 효과(HTE) 추정 전략

---

### **핵심 아이디어**
- **목표의 전환**: "정책의 **평균** 효과는 얼마인가? (ATE)"에서 "특정 특성($X$)을 가진 개인에게 정책의 효과는 얼마인가? (**CATE**, Conditional Average Treatment Effect)"로 질문을 전환한다.
    - CATE($X_i$) = $E[Y_i(1) - Y_i(0) | X=X_i]$
- **Meta-Learner란?**: CATE를 추정하기 위해 특별히 고안된 새로운 머신러닝 알고리즘이 아니라, 기존의 어떤 머신러닝 예측 모델(라쏘, 랜덤 포레스트 등)이든 **'기반 학습기(base learner)'**로 사용하여 CATE를 추정하는 **'전략' 또는 '레시피'**를 의미한다.
- **비유**: Meta-learner는 스스로 요리를 하지 않는 '마스터 셰프'와 같다. 대신, 여러 '일반 셰프'(base learner)들에게 "너는 이 재료만 써서 요리해", "너는 저 재료만 써서 요리해", "둘의 요리를 합쳐서 새로운 맛을 내" 와 같이 지시하는 **'상위 레벨의 전략'**을 제공한다.

<br>

### **주요 Meta-Learner 전략**
CATE를 추정하기 위해, 우리는 두 가지 핵심적인 '성가신 함수(nuisance function)'를 예측해야 한다:
1.  **결과 모델 (Outcome Model)**: $E[Y|X, D]$
2.  **처치 모델 (Treatment Model / 성향점수)**: $P(D=1|X)$

어떤 Meta-learner를 사용하느냐는, 이 함수들을 어떤 머신러닝 모델로, 어떤 순서와 방식으로 조합할지를 결정한다.

#### **1. S-Learner (Single-Learner)**
- **전략**: 가장 단순한 방법. 처치 여부($D$)를 다른 통제변수($X$)들과 똑같은 **일반적인 독립변수 중 하나**로 취급하여, **단 하나의(Single) 머신러닝 모델**을 만든다.
    - $Y \sim D + X_1 + X_2 + \dots$
- **CATE 추정**: 특정 개인($X_i$)의 CATE를 알기 위해, 이 모델에 해당 개인의 데이터를 두 번 입력한다.
    1. 처치를 받았다고 가정($D=1$)하고 예상 결과($\hat{Y}_1$)를 예측한다.
    2. 처치를 받지 않았다고 가정($D=0$)하고 예상 결과($\hat{Y}_0$)를 예측한다.
    - $CATE(X_i) = \hat{Y}_1 - \hat{Y}_0$
- **단점**: 처치 변수($D$)의 효과가 다른 변수들에 비해 작을 경우, 모델이 $D$ 변수를 거의 무시해버려 HTE를 제대로 잡아내지 못할 위험이 있다.

#### **2. T-Learner (Two-Learners)**
- **전략**: 가장 직관적인 방법. 데이터를 **처치 그룹과 통제 그룹으로 분리**한 뒤, **두 개의(Two) 독립적인 머신러닝 모델**을 각각 학습시킨다.
    - 모델 1 (처치 그룹): $Y_1 \sim X_1 + X_2 + \dots$
    - 모델 0 (통제 그룹): $Y_0 \sim X_1 + X_2 + \dots$
- **CATE 추정**: 특정 개인($X_i$)에 대해, 두 모델의 예측값 차이를 계산한다.
    - $CATE(X_i) = \text{model}_1.\text{predict}(X_i) - \text{model}_0.\text{predict}(X_i)$
- **단점**: 데이터를 둘로 나누므로 학습 데이터의 양이 줄어든다. 특히 한 그룹이 다른 그룹보다 매우 작을 경우(예: 희귀병 치료), 해당 그룹의 모델 성능이 매우 나빠질 수 있다.

#### **3. X-Learner & R-Learner (Advanced Learners)**
- **X-Learner**: T-Learner의 단점(특히 그룹 크기가 불균형할 때)을 보완하기 위해 고안된 다단계 학습 전략. 한 그룹의 모델로 다른 그룹의 반사실적 가상을 예측하고, 그 예측 오차를 다시 모델링하는 등 더 정교한 방식을 사용한다.
- **R-Learner**: **이론적으로 가장 정교한 Meta-learner 중 하나.** DML과 매우 유사한 아이디어에 기반한다.
    - **핵심 원리**: $Y$와 $D$ 각각에서 $X$의 효과를 제거한 **'잔차(residual)'** 사이의 관계를 모델링하여 CATE($\tau(X)$)를 직접 추정한다.
        $$
        (Y - E[Y|X]) = \tau(X) \cdot (D - E[D|X]) + \text{error}
        $$
    - **장점**: DML처럼 **나이만-직교성(Neyman Orthogonality)** 원리를 사용하여, 예측 모델($E[Y|X]$와 $E[D|X]$)에 약간의 오류가 있더라도 최종 CATE 추정치($\tau(X)$)는 비교적 편향이 적도록 설계되어 있다.

<br>

### **결론**
- Meta-learner는 특정 알고리즘의 이름이 아니라, **HTE를 추정하기 위한 다양한 '전략'의 집합**이다.
- 단순한 S-learner, T-learner부터 이론적으로 정교한 X-learner, R-learner까지 다양한 옵션이 존재하며, 연구 상황에 맞는 적절한 전략을 선택하는 것이 중요하다.
.

# 33. 트리 기반 HTE 추정: 일반화 랜덤 포레스트와 인과 포레스트

---

### **1. 일반화 랜덤 포레스트 (Generalized Random Forests, GRF)**

- **핵심 아이디어**: 기존의 랜덤 포레스트는 **'조건부 평균($E[Y|X]$)'**이라는 단 하나의 통계적 목표를 예측하기 위해 설계되었다. GRF는 이 아이디어를 확장하여, **연구자가 원하는 거의 모든 종류의 통계적 목표($\theta(X)$)**를 추정할 수 있도록 랜덤 포레스트를 **'일반화(generalize)'**한 이론적 프레임워크이다.
- **비유**: 기본 랜덤 포레스트가 '사과'만 담을 수 있는 바구니라면, GRF는 연구자가 원하는 어떤 과일(인과 효과, 분위수 등)이든 담을 수 있도록 **모양을 바꾸는 '만능 바구니'**와 같다.
- **가장 중요한 변화: 분기 기준(Splitting Rule)의 일반화**
    - 기존 랜덤 포레스트: 예측 오차(MSE)를 가장 많이 줄이는 방향으로 나무를 분기한다.
    - **GRF**: 연구자가 정의한 **목표($\theta(X)$)의 '이질성(heterogeneity)'이 가장 커지는 방향**으로 나무를 분기한다. 즉, 분기 후 두 자식 노드(child nodes) 간의 목표값 차이가 최대화되도록 한다.

<br>

### **2. 인과 포레스트 (Causal Forest)**

- **정의**: **GRF라는 일반적인 프레임워크를 '조건부 평균 처치 효과(CATE)' 추정이라는 특정 목적에 맞게 구현한, 가장 성공적이고 대표적인 모델.**
- **핵심 원리**:
    - **목표 설정**: 인과 포레스트의 목표($\theta(X)$)는 바로 **CATE($\tau(X)$)**이다.
    - **새로운 분기 기준**: 따라서 인과 포레스트의 의사결정나무는, 분기했을 때 **두 자식 노드 간의 '처치 효과(treatment effect)' 차이가 가장 커지는** 질문을 찾아 분기한다.
    - **결과**: 이렇게 만들어진 숲은, 비슷한 특성을 가진 개체들을 같은 잎(leaf)으로 모으는 것이 아니라, **'정책에 비슷하게 반응하는(비슷한 처치 효과를 갖는)'** 개체들을 같은 잎으로 모으게 된다.

<br>

### **인과 포레스트의 작동 과정 (GRF의 구체화)**
인과 포레스트는 단순한 예측을 넘어 인과 효과를 '편향 없이' 추정하기 위해 몇 가지 정교한 장치를 추가로 사용한다.

1.  **잔차화 (Orthogonalization / Residualization)**
    - **목표**: DML과 마찬가지로, 교란 변수($X$)가 Y와 D 모두에 미치는 영향을 제거하여, 처치 효과($\tau$)에 대한 추정이 교란 변수에 의해 오염되는 것을 막는다.
    - **과정**: DML의 1, 2단계와 유사하게, 별도의 머신러닝 모델을 사용해 $Y$와 $D$ 각각에서 $X$의 효과를 예측하고 그 잔차($\tilde{Y}, \tilde{D}$)를 구한다. 인과 포레스트는 이 '깨끗해진' 잔차들을 가지고 나무를 성장시킨다.

2.  **정직한 추정 (Honest Estimation)과 교차적합 (Cross-fitting)**
    - **문제**: 같은 데이터를 사용해서 나무의 '구조를 만들고(분기)' 동시에 그 나무의 '결과를 채우면(잎의 값 추정)', 과적합 편향이 발생할 수 있다.
    - **해법**: 데이터를 두 그룹(A, B)으로 나눈다.
        - **A 그룹 데이터**: 오직 나무의 **'구조를 만드는 데(splitting)'**만 사용한다.
        - **B 그룹 데이터**: 이렇게 만들어진 나무의 각 잎(leaf)에 **'값을 채우는 데(estimation)'**만 사용한다.
    - **효과**: 나무의 구조를 결정하는 정보와 그 구조 안에서 효과를 추정하는 정보를 분리하여, 과적합을 방지하고 통계적으로 유의미한 신뢰구간을 계산할 수 있게 해준다.

<br>

### **결론**
- **GRF**는 랜덤 포레스트를 다양한 통계적 추정 문제에 적용할 수 있도록 만든 **'이론적 틀'**이다.
- **인과 포레스트**는 그 틀을 이용하여, **이질적 처치 효과(HTE)를 추정**하는 데 특화된 매우 강력하고 실용적인 알고리즘이다.
- 잔차화, 정직한 추정 등의 장치를 통해, 머신러닝의 강력한 예측력을 활용하면서도 신뢰도 높은 인과적 추론을 가능하게 만들어준다.

# 34. 딥러닝과 인과추론 (Deep Learning & Causal Inference)

---

### **핵심 아이디어**
- 지금까지 다룬 모든 방법론은 기본적으로 **'정형 데이터(structured data)'**, 즉 엑셀 표처럼 깔끔하게 정리된 숫자나 카테고리 형태의 데이터를 가정했다.
- 하지만 현실 세계의 많은 정보는 **이미지, 텍스트, 소셜 네트워크**와 같은 복잡한 비정형 데이터 안에 담겨있다.
- **딥러닝과 인과추론**은, 딥러닝 모델(신경망)의 강력한 **'표현 학습(representation learning)'** 능력을 활용하여, 이러한 비정형 데이터로부터 인과 효과를 추정하려는 최첨단 연구 분야이다.
- **비유**: DML이 수백 개의 '숫자' 통제변수를 처리했다면, 딥러닝 인과추론은 수만 개의 픽셀로 이루어진 '광고 이미지' 자체가 처치 변수($D$)가 되거나, 장문의 '뉴스 기사'가 통제변수($X$)가 되는 문제를 다룬다.

<br>

### **딥러닝은 인과추론에서 어떤 역할을 하는가?**
딥러닝은 인과추론의 여러 단계에서 '강력한 예측 엔진'으로서의 역할을 수행한다.

1.  **고차원 통제변수 처리 (High-Dimensional Confounders)**
    - **문제**: 통제변수($X$)가 텍스트나 이미지일 경우, 그 차원은 수만, 수백만 개에 달한다. 라쏘나 랜덤 포레스트만으로는 이 정보를 효과적으로 압축하고 처리하기 어렵다.
    - **딥러닝의 역할**: 딥러닝 모델(예: 오토인코더, CNN, RNN)은 이러한 고차원의 비정형 데이터로부터 핵심적인 특징(feature)을 자동으로 추출하여, 저차원의 유의미한 **'표현(representation)'** 벡터로 압축한다. 이 압축된 벡터를 DML이나 Meta-learner의 통제변수로 사용하여 OVB 문제를 해결한다.

2.  **비정형 처치 변수 (Unstructured Treatments)**
    - **문제**: 처치($D$) 자체가 이미지나 텍스트인 경우. (예: "A 타입 광고 이미지가 B 타입 광고 이미지보다 클릭률에 미치는 인과적 효과는?")
    - **딥러NING의 역할**: 딥러닝은 각 이미지나 텍스트를 벡터 공간의 한 점으로 표현한다. 이 벡터 표현을 이용해 처치 효과를 모델링하고, 특정 이미지의 어떤 '특징'(예: 모델의 미소, 배경색)이 효과에 영향을 미쳤는지 분석할 수 있다.

3.  **반사실적 가상 생성 (Counterfactual Generation)**
    - **가장 흥미로운 분야.**
    - **아이디어**: "만약 이 사람의 엑스레이 사진에 질병이 '없었다면' 어떻게 보였을까?"와 같은 **'반사실적 이미지'**를 딥러닝 생성 모델(Generative Models)을 이용해 직접 만들어내는 시도.
    - **주요 기법**:
        - **Causal GAN (Generative Adversarial Networks)**: 생성 모델인 GAN을 인과추론에 적용하여, 관찰된 데이터로부터 반사실적 결과를 생성하고 이를 비교하여 인과 효과를 추정한다.
        - **VAE (Variational Autoencoders)**: VAE를 기반으로 잠재 공간(latent space)에서 인과적 구조를 학습하려는 시도. (예: CEVAE)

<br>

### **주요 기법 및 모델**
- **Dragonnet / TARNet**: 이질적 처치 효과(HTE)를 추정하기 위한 대표적인 초기 딥러닝 아키텍처. 신경망의 일부는 처치 그룹과 통제 그룹이 공유하고, 끝부분만 분리하여 각 그룹의 결과를 예측하는 효율적인 구조를 제안했다.
- **CEVAE (Causal Effect Variational Autoencoder)**: 이미지와 같은 고차원 데이터에서 숨겨진 교란 변수(confounder)가 있을 때, VAE를 이용하여 이를 모델링하고 인과 효과를 추정하려는 시도.

<br>

### **결론 및 한계**
- **장점**:
    - **적용 범위의 확장**: 전통적인 계량경제학이 다루지 못했던 이미지, 텍스트 등 비정형 데이터를 인과추론의 영역으로 끌어왔다.
    - **강력한 성능**: 매우 복잡한 비선형 관계나 상호작용을 자동으로 학습하는 데 탁월하다.

- **한계 및 현재 연구 방향**:
    - **블랙박스 문제**: 왜 그런 결과가 나왔는지 '해석'하기가 매우 어렵다.
    - **강력한 가정 필요**: 딥러닝 모델이 인과 효과를 올바르게 식별하기 위해서는, 관찰 불가능한 교란 변수가 없다는(unconfoundedness) 가정이 더 강력하게 요구된다.
    - **이론적 기반**: 아직 많은 방법론들이 실험적으로는 잘 작동하지만, 통계적인 이론 기반이 DML이나 인과 포레스트만큼 견고하게 확립되지 않은 경우가 많다. **현재 가장 활발하게 연구가 진행 중인 최전선 분야이다.**

# 35. 인과적 발견 (Causal Discovery)

---

### **핵심 아이디어**
- **목표의 전환**: "X가 Y에 미치는 효과의 크기는 얼마인가?" (Causal Inference) 라는 질문에서, **"어떤 변수가 원인이고 어떤 변수가 결과인가? 변수들 간의 인과관계 '구조(structure)'는 어떻게 생겼는가?"** (Causal Discovery) 라는 더 근본적인 질문으로 전환한다.
- **목표**: 데이터로부터 변수들 간의 **인과관계를 나타내는 방향 그래프(Directed Acyclic Graph, DAG)**, 즉 '인과 지도'를 역으로 추론해내는 것을 목표로 한다.
- **비유**: 지금까지의 인과추론이 'A 의사가 B 환자에게 특정 약을 투여했을 때의 효과'를 측정하는 것이었다면, 인과적 발견은 수많은 환자들의 데이터를 보고 **"어떤 유전자가 어떤 질병의 원인인지" 그 '원인과 결과의 연결망' 자체를 찾아내는 것**과 같다.

<br>

### **어떻게 인과관계를 '발견'하는가?: 핵심 가정**
데이터의 '상관관계'로부터 '인과관계'의 방향을 추론하기 위해서는 매우 강력한 가정이 필요하다.

1.  **인과적 마르코프 조건 (Causal Markov Condition)**
    - **가정**: 어떤 변수는, 자신의 **'직계 원인(parents)'**들의 값이 주어지면, 자신의 '후손(descendants)'을 제외한 다른 모든 변수들과 통계적으로 독립이 된다.
    - **더 쉬운 설명**: "나의 상태는 나의 직계 부모에 의해서만 결정되며, 내 조상이나 사촌의 상태는 부모를 통해 전달된 정보 외에는 추가적인 정보를 주지 않는다."
    - **예시**: `흡연 -> 타르 축적 -> 폐암`. 여기서 '타르 축적'의 정도가 주어지면, '폐암'에 걸릴 확률은 '흡연' 여부와는 조건부 독립이 된다. 즉, 흡연은 타르 축적을 통해서만 폐암에 영향을 미친다.

2.  **성실성 가정 (Faithfulness Condition)**
    - **가정**: 데이터에서 관찰되는 **'모든' 조건부 독립 관계는 인과 그래프 구조에 의해서만 발생**한다.
    - **더 쉬운 설명**: "우연히, 서로 다른 인과 경로의 효과가 정확히 상쇄되어, 실제로는 관계가 있는데도 데이터상에서는 독립처럼 보이는 '우연한 일치'는 일어나지 않는다."
    - **예시**: 약을 먹으면 병이 낫지만($+$ 효과), 동시에 부작용으로 다른 문제가 생겨($-$ 효과) 최종적인 건강 상태에 아무런 변화가 없는 것처럼 보이는, 그런 우연은 없다고 가정하는 것이다.

<br>

### **주요 알고리즘**

이러한 가정을 바탕으로, 데이터의 조건부 독립 관계를 테스트하여 인과 그래프를 찾아내는 여러 알고리즘이 개발되었다.

- **제약 기반 알고리즘 (Constraint-based Algorithms)**:
    - **PC 알고리즘, FCI 알고리즘**: 변수들 간의 조건부 독립성을 반복적으로 테스트하여, 인과관계가 '없는' 연결들을 하나씩 지워나가는 방식으로 그래프를 만든다.

- **점수 기반 알고리즘 (Score-based Algorithms)**:
    - 가능한 모든 그래프 구조에 대해 '데이터를 얼마나 잘 설명하는지' 점수를 매기고, 가장 높은 점수를 받는 그래프를 최적의 인과 구조로 선택한다.

- **구조적 인과 모형 (Structural Causal Models, SCM)**:
    - 각 변수가 자신의 직계 원인에 의해 어떻게 결정되는지를 나타내는 **함수 방정식들의 집합**으로 인과 시스템을 표현한다. 딥러닝과 결합하여 비선형적인 관계를 다루는 연구가 활발하다.

<br>

### **결론 및 경제학에서의 위치**
- **장점**:
    - **가설 생성**: 기존에 알려지지 않았던 새로운 인과관계 가설을 데이터로부터 직접 생성해낼 수 있다.
    - **근본적 이해**: 시스템 전체의 작동 방식을 구조적으로 이해하는 데 도움을 준다.

- **한계**:
    - **강력한 가정**: 위에서 설명한 가정들이 현실에서 성립하는지 보장하기 어렵다.
    - **데이터 요구량**: 신뢰도 높은 인과 그래프를 추론하기 위해서는 매우 많은 양의 데이터가 필요하다.
    - **경제학에서의 위치**: 아직까지는 DID, IV 등 '처치 효과'를 추정하는 방법론들에 비해 주류로 사용되지는 않는다. 하지만, 머신러닝의 발전과 함께 그 중요성이 점점 더 커지고 있는, **가장 도전적이고 근본적인 최전선 연구 분야**이다.